# MNIST Classification with Logistic Regression

## Agenda:
1. Tensorflow
2. MNIST data
	- Collect data with one_hot
	- Get train and test data
3. Define Parameters
4. Build the Graph
	- Define placeholders and Vriables
	- Prediction
	- Loss
	- Optimizer
	- Model Accuracy
	- Logging
5. Execute the Graph
	- Create Sesson
	- Run session on training set
	- Run session on Test set
	- Log the loss and accuracy for train and test
	- Sabe the model
	- Close the session

## TensorFlow
is an open-source software library for dataflow programming across a range of tasks. It is a symbolic math library, mostly used for machine learning applications such as neural networks. Its developed by Google.

## MNIST data
The MNIST database of handwritten digits, available from http://yann.lecun.com/exdb/mnist/ page, has a training set of 55,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image of 28x28.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

Four files are available on this database:    
train-images-idx3-ubyte.gz:  training set images  
train-labels-idx1-ubyte.gz:  training set labels   
t10k-images-idx3-ubyte.gz:   test set images    
t10k-labels-idx1-ubyte.gz:   test set labels 

28x28 handwritten digits image data faltten in to a vector form of size 784

In [1]:
import tensorflow as tf

In [2]:
#Reset tensorflow graph
tf.reset_default_graph()

### Collect Data
Use MNIST data provided with tensorflow library.   
**One-Hot encoding:** where we create N new features, where N is the number of unique values in the original feature. In our exampel N would be equal to 10, because we have 10 digits(0 to 9). Each of these features be binary and would correspond to one of 0-9 digit. In our example first feature value in trainY set is 7 and it is equal to [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.] in one hot encoding.

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

Get Training and Test Data

In [4]:
trainX = mnist.train.images
trainY = mnist.train.labels

In [5]:
testX = mnist.test.images
testY = mnist.test.labels

How many Training and Test Examples?

In [6]:
print('Training input features: ', trainX.shape)
print('Training target data: ', trainY.shape)

Training input features:  (55000, 784)
Training target data:  (55000, 10)


In [7]:
# First value in trainY set 7 is equal to [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.] 
trainY[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [8]:
print('Test input features: ', testX.shape)
print('Test target data: ', testY.shape)

Test input features:  (10000, 784)
Test target data:  (10000, 10)


## Lets define some parameters

In [9]:
#Directory to save logs and graph, change it as needed
logs_path='/tmp/mnist/lr1'

#Learning rate
learning_rate = 0.03

#Number of input features - MNIST has 784 features
n_features = trainX.shape[1]

#Number of possible output classess - 10 for MNIST
n_classes = trainY.shape[1]

#Model name for storage
model_name = 'mnist_lr.ckpt'

#How many times all the data will be shown to model
training_epochs = 100

# Build the Graph

Input placeholders

In [10]:
with tf.name_scope('input'):
    
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

Define Weights and Bias

In [11]:
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([n_features, n_classes]))  #784,10
    b = tf.Variable(tf.zeros([n_classes])) #10

Prediction    

In [12]:
with tf.name_scope("Output"):
    y = tf.nn.softmax(tf.matmul(x,W) + b)

Loss

In [13]:
with tf.name_scope('Loss'):
    #Cross Entropy Loss
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

GradientDescent Optimizer

In [14]:
with tf.name_scope('train'):    
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        cross_entropy)    

Model Accuracy

In [15]:
with tf.name_scope('Accuracy'):
    
    #Get the number with highest probability
    prediction = tf.argmax(y,1,name="Predict")
    
    #Compare prediction with actual
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    
    #Calculate mean accuracy across all data examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                              name="accuracy")

Loss and Accuracy Logging

In [16]:
# Training loss and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)

#Test loss and accuracy
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

# create log writer object
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

# Execute the Graph

In [17]:
#Start Graph execution
with tf.Session() as sess:
    
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # perform training cycles
    for epoch in range(training_epochs):
        
        # perform the operations we defined earlier on batch
        _,acc,loss = sess.run([train_op, training_accuracy,training_loss],
                              feed_dict={x: trainX, y_: trainY})
        
        #log training accuracy and loss
        writer.add_summary(acc, epoch)
        writer.add_summary(loss, epoch) 
        
        #Test loss and accuracy
        #Please note we are giving test data for session run
        acc,loss = sess.run([test_accuracy,test_loss],
                                   feed_dict={x: testX, y_: testY})
        
        writer.add_summary(acc, epoch)
        writer.add_summary(loss, epoch)
        
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
            print ("Test Accuracy: ", accuracy.eval(feed_dict={x: testX, 
                                                               y_: testY}))               
    #Create a Saver to save the graph
    saver = tf.train.Saver()
    saver.save(sess, logs_path + '/' + model_name)

Epoch:  0
Test Accuracy:  0.6705
Epoch:  5
Test Accuracy:  0.7134
Epoch:  10
Test Accuracy:  0.7383
Epoch:  15
Test Accuracy:  0.7545
Epoch:  20
Test Accuracy:  0.7638
Epoch:  25
Test Accuracy:  0.7775
Epoch:  30
Test Accuracy:  0.7875
Epoch:  35
Test Accuracy:  0.7946
Epoch:  40
Test Accuracy:  0.7997
Epoch:  45
Test Accuracy:  0.8046
Epoch:  50
Test Accuracy:  0.8087
Epoch:  55
Test Accuracy:  0.8135
Epoch:  60
Test Accuracy:  0.8161
Epoch:  65
Test Accuracy:  0.8187
Epoch:  70
Test Accuracy:  0.8216
Epoch:  75
Test Accuracy:  0.8243
Epoch:  80
Test Accuracy:  0.8269
Epoch:  85
Test Accuracy:  0.8285
Epoch:  90
Test Accuracy:  0.8318
Epoch:  95
Test Accuracy:  0.8332


In [18]:
# Close the session
sess.close()